In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/TY_GRP_4/SI/si2.csv',error_bad_lines = False)
df.head()

FileNotFoundError: ignored

In [ ]:
top_ten = df[df['Total Rating Count'] > 50]
top_ten.sort_values(by='average_rating', ascending=False)
plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(10, 10))
data = top_ten.sort_values(by='average_rating', ascending=False).head(10)
sns.barplot(x="average_rating", y="Company", data=data, palette='inferno')

KeyError: ignored

In [ ]:
most_rated = df.sort_values('Total Rating Count', ascending = False).head(10).set_index('Company')
plt.figure(figsize=(15,10))
ax = sns.barplot(most_rated['Total Rating Count'], most_rated.index, palette = 'inferno')
totals = []
for i in ax.patches:
    totals.append(i.get_width())
total = sum(totals)
for i in ax.patches:
    ax.text(i.get_width()+.2, i.get_y()+.2,str(round(i.get_width())), fontsize=15,color='black')
plt.show()

In [ ]:
df.average_rating = df.average_rating.astype(float)
fig, ax = plt.subplots(figsize=[15,10])
sns.distplot(df['average_rating'],ax=ax)
ax.set_title('Average rating distribution for all books',fontsize=20)
ax.set_xlabel('Average rating',fontsize=13)

In [ ]:
# ax = sns.relplot(data=df, x="average_rating", y="ratings_count", color = 'red', sizes=(100, 200), height=7, marker='o')
# plt.title("Relation between Rating counts and Average Ratings",fontsize = 15)
# ax.set_axis_labels("Average Rating", "Ratings Count")
df2 = df.copy()

In [ ]:
df2.loc[ (df2['average_rating'] >= 0) & (df2['average_rating'] <= 1), 'rating_between'] = "between 0 and 1"
df2.loc[ (df2['average_rating'] > 1) & (df2['average_rating'] <= 2), 'rating_between'] = "between 1 and 2"
df2.loc[ (df2['average_rating'] > 2) & (df2['average_rating'] <= 3), 'rating_between'] = "between 2 and 3"
df2.loc[ (df2['average_rating'] > 3) & (df2['average_rating'] <= 4), 'rating_between'] = "between 3 and 4"
df2.loc[ (df2['average_rating'] > 4) & (df2['average_rating'] <= 5), 'rating_between'] = "between 4 and 5"

In [ ]:
df2

In [ ]:
rating_df = pd.get_dummies(df2['rating_between'])
company_df = pd.get_dummies(df2['Company'])

In [ ]:
features = pd.concat([rating_df, 
                      company_df, 
                      df2['average_rating'], 
                      df2['Total Rating Count']], axis=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
features = min_max_scaler.fit_transform(features)

In [ ]:

model = neighbors.NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
model.fit(features)
dist, idlist = model.kneighbors(features)

In [ ]:
def BookRecommender(book_name):
    book_list_name = []
    book_id = df2[df2['Company'] == book_name].index
    book_id = book_id[0]
    for newid in idlist[book_id]:
        book_list_name.append(df2.loc[newid].Company)
    return book_list_name
    
BookNames = BookRecommender('Turkish Airlines(Economy Class)')
BookNames

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
data = pd.read_csv("si2.csv")
data.head()

In [ ]:
data.columns

In [ ]:
data.isna().sum()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.isna().sum()

In [ ]:
# data = data.copy()
# scaler = StandardScaler()
# features = [['overall', 'seat_comfort', 'cabin_service', 'food_bev', 'entertainment','ground_service','value_for_money']]
# for feature in features:
#     data[feature] = scaler.fit_transform(data[feature])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
corr_Pearson = data.corr(method='pearson')

figure = plt.figure(figsize=(25,15))
sns.heatmap(corr_Pearson,vmin=-1,vmax=+1,cmap='Oranges',annot=True, 
            linewidths=2,linecolor = 'black')
plt.title('Pearson Correlation')
plt.show()

In [ ]:
#for economy class maximum overall rating is 0.991,1.9
import seaborn as sns
sns.countplot(x='cabin', hue=pd.cut(data['overall'],10),data=data)
plt.legend(loc='upper right', title='o')

In [ ]:
plt.figure(figsize=(8,5))
sns.set(style="darkgrid")
ax = sns.countplot(x="date_flown", data=data, palette="Set2", order=data['date_flown']
                   .value_counts().index[0:15])

In [ ]:
data.groupby('date_flown')['airline'].agg('count').sort_values()

In [ ]:
cabin_spread = data.groupby("cabin")["overall"].count().reset_index().sort_values(by="overall", ascending=False).head(10)
cabin_spread.head(10)

In [ ]:
ratings = pd.DataFrame(data.groupby('airline')['overall'].mean())
ratings.head()

In [ ]:
airline_counts = pd.DataFrame(data["airline"].value_counts())
print(airline_counts)
airline_counts.sort_values("airline", ascending=False)

In [ ]:
# list of numerical variables
numerical_features = [feature for feature in data.columns if data[feature].dtypes != 'O']
print('Number of numerical variables: ', len(numerical_features))
data[numerical_features].head()# visualise the numerical variables

In [ ]:
data['id_cus'] = data.groupby(['author']).ngroup()

In [ ]:
data['airline_id'] = data[['airline']]

In [ ]:
col = ['airline_id']
le= LabelEncoder()
data[col] = data[col].apply(le.fit_transform)

In [ ]:
airline_features_df=data.pivot_table(index='airline',columns='id_cus',values='overall').fillna(0)
airline_features_df.head()

In [ ]:
# airline_features_df1=data.pivot_table(index='airline',columns='id_cus',values='food_bev').fillna(0)
# airline_features_df1.head()

In [ ]:
#for getting faster result used csr
from scipy.sparse import csr_matrix
airline_features_df_matrix = csr_matrix(airline_features_df.values)
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(airline_features_df_matrix)

In [ ]:
query_index = np.random.choice(airline_features_df.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(airline_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

In [ ]:
# function is used to get a copy of an given array collapsed into one dimension.
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(airline_features_df.index[query_index]))
    else:
        print('{0}: {1}'.format(i, airline_features_df.index[indices.flatten()[i]], distances.flatten()[i]))
        

In [ ]:
# from scipy.sparse import csr_matrix
# airline_features_df_matrix = csr_matrix(airline_features_df.values)
# from sklearn.neighbors import NearestNeighbors
# model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
# model_knn.fit(airline_features_df_matrix)
# y_pred=model_knn.predict(airline_features_df_matrix)
# print("Accuracy: ",accuracy_score(airline_features_df_matrix))

In [ ]:
col = data.columns.tolist()
le= LabelEncoder()
data[col] =data[col].apply(le.fit_transform)

In [ ]:
from sklearn.datasets import make_classification
X = data.drop(['recommended'], axis=1)
y = data['recommended']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size = 0.05, random_state = 42)

In [ ]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score
model = KNeighborsClassifier(metric = 'cosine', algorithm = 'brute')
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print("Accuracy: ",accuracy_score(y_test,y_pred))